# Импорт библиотек

In [1]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import datatable as dt
import xgboost as xgb

from scipy.stats import shapiro # Для проверки однородности рядов. Проверка данных на нормальность. Тест нормальности (тест Шапиро-Уилка)
from scipy.stats import kstest # Тест Колмогорова-Смирнова
from scipy import stats
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score,fbeta_score, precision_score,recall_score,accuracy_score, roc_auc_score
from sklearn.utils import resample
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.impute import SimpleImputer # для замены пропусков либо константными значениями, либо же статистическими (средним или медианой)
from sklearn.impute import KNNImputer # для замены пропусков ближайшими соседями 
from sklearn.model_selection import cross_val_score


In [2]:
df = pd.read_csv('sup_data/summary_df.csv')
df.head()

,rn,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,pre_since_opened_9,pre_since_opened_10,pre_since_opened_11,pre_since_opened_12,pre_since_opened_13,pre_since_opened_14,pre_since_opened_15,pre_since_opened_16,pre_since_opened_17,pre_since_opened_18,pre_since_opened_19,pre_since_confirmed_0,pre_since_confirmed_1,pre_since_confirmed_2,pre_since_confirmed_3,pre_since_confirmed_4,pre_since_confirmed_5,pre_since_confirmed_6,pre_since_confirmed_7,pre_since_confirmed_8,pre_since_confirmed_9,pre_since_confirmed_10,pre_since_confirmed_11,pre_since_confirmed_12,pre_since_confirmed_13,pre_since_confirmed_14,pre_since_confirmed_15,pre_since_confirmed_16,pre_since_confirmed_17,pre_pterm_0,pre_pterm_1,pre_pterm_2,pre_pterm_3,pre_pterm_4,pre_pterm_5,pre_pterm_6,pre_pterm_7,pre_pterm_8,pre_pterm_9,pre_pterm_10,pre_pterm_11,pre_pterm_12,pre_pterm_13,pre_pterm_14,pre_pterm_15,pre_pterm_16,pre_pterm_17,pre_fterm_0,pre_fterm_1,pre_fterm_2,pre_fterm_3,pre_fterm_4,pre_fterm_5,pre_fterm_6,pre_fterm_7,pre_fterm_8,pre_fterm_9,pre_fterm_10,pre_fterm_11,pre_fterm_12,pre_fterm_13,pre_fterm_14,pre_fterm_15,pre_fterm_16,pre_till_pclose_0,pre_till_pclose_1,pre_till_pclose_2,pre_till_pclose_3,pre_till_pclose_4,pre_till_pclose_5,pre_till_pclose_6,pre_till_pclose_7,pre_till_pclose_8,pre_till_pclose_9,pre_till_pclose_10,pre_till_pclose_11,pre_till_pclose_12,pre_till_pclose_13,pre_till_pclose_14,pre_till_pclose_15,pre_till_pclose_16,pre_till_fclose_0,pre_till_fclose_1,pre_till_fclose_2,pre_till_fclose_3,pre_till_fclose_4,pre_till_fclose_5,pre_till_fclose_6,pre_till_fclose_7,pre_till_fclose_8,pre_till_fclose_9,pre_till_fclose_10,pre_till_fclose_11,pre_till_fclose_12,pre_till_fclose_13,pre_till_fclose_14,pre_till_fclose_15,pre_loans_credit_limit_0,pre_loans_credit_limit_1,pre_loans_credit_limit_2,pre_loans_credit_limit_3,pre_loans_credit_limit_4,pre_loans_credit_limit_5,pre_loans_credit_limit_6,pre_loans_credit_limit_7,pre_loans_credit_limit_8,pre_loans_credit_limit_9,pre_loans_credit_limit_10,pre_loans_credit_limit_11,pre_loans_credit_limit_12,pre_loans_credit_limit_13,pre_loans_credit_limit_14,pre_loans_credit_limit_15,pre_loans_credit_limit_16,pre_loans_credit_limit_17,pre_loans_credit_limit_18,pre_loans_credit_limit_19,pre_loans_next_pay_summ_0,pre_loans_next_pay_summ_1,pre_loans_next_pay_summ_2,pre_loans_next_pay_summ_3,pre_loans_next_pay_summ_4,pre_loans_next_pay_summ_5,pre_loans_next_pay_summ_6,pre_loans_outstanding_1,pre_loans_outstanding_2,pre_loans_outstanding_3,pre_loans_outstanding_4,pre_loans_outstanding_5,pre_loans_total_overdue_0,pre_loans_total_overdue_1,pre_loans_max_overdue_sum_0,pre_loans_max_overdue_sum_1,pre_loans_max_overdue_sum_2,pre_loans_max_overdue_sum_3,pre_loans_credit_cost_rate_0,pre_loans_credit_cost_rate_1,pre_loans_credit_cost_rate_2,pre_loans_credit_cost_rate_3,pre_loans_credit_cost_rate_4,pre_loans_credit_cost_rate_5,pre_loans_credit_cost_rate_6,pre_loans_credit_cost_rate_7,pre_loans_credit_cost_rate_8,pre_loans_credit_cost_rate_9,pre_loans_credit_cost_rate_10,pre_loans_credit_cost_rate_11,pre_loans_credit_cost_rate_12,pre_loans_credit_cost_rate_13,pre_loans5_0,pre_loans5_1,pre_loans5_2,pre_loans5_3,pre_loans5_5,pre_loans5_6,pre_loans5_7,pre_loans5_8,pre_loans5_9,pre_loans5_10,pre_loans5_11,pre_loans5_13,pre_loans5_16,pre_loans530_0,pre_loans530_1,pre_loans530_2,pre_loans530_3,pre_loans530_4,pre_loans530_5,pre_loans530_6,pre_loans530_7,pre_loans530_8,pre_loans530_9,pre_loans530_10,pre_loans530_11,pre_loans530_12,pre_loans530_13,pre_loans530_14,pre_loans530_15,pre_loans530_16,pre_loans530_17,pre_loans530_18,pre_loans530_19,pre_loans3060_0,pre_loans3060_1,pre_loans3060_2,pre_loans3060_3,pre_loans3060_4,pre_loans3060_5,pre_loans3060_6,pre_loans3060_7,pre_loans3060_8,pre_loans3060_9,pre_loans6090_0,pre_loans6090_1,pre_loans6090_2,pre_loans6090_3,pre_loans6090_4,pre_loans90_2,pre_loans90_3,pre_loans90_8,pre_loans

## Приводим датасет к типу данных "int8" для уменьшения занимаемой памяти

In [3]:
df = df.astype('int8')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Columns: 421 entries, rn to flag
dtypes: int8(421)
memory usage: 1.2 GB


# Перейдем к обучению модели.

Для начала создадим матрицу объект-признак, Х. -> (Все признаки). 

у - вектор целевой перееменной.(наличие дефолта)

In [19]:
X = df.drop(columns=['flag'])
y = df['flag']

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Columns: 420 entries, rn to fclose_flag_1
dtypes: int8(420)
memory usage: 1.2 GB


## Разделим датасет на тренировочную и тестовую выборки:

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=313)

In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2400000 entries, 1662644 to 2149896
Columns: 420 entries, rn to fclose_flag_1
dtypes: int8(420)
memory usage: 979.6 MB


In [10]:
X_train.shape

(2400000, 420)

## Моделирование 

### CatBoostClassifier базовая

In [70]:
%%time
from catboost import CatBoostClassifier
catboost = CatBoostClassifier(random_state=313)
catboost.fit(X_train, y_train)

# Получение вероятностей для класса 1
pred_proba = catboost.predict_proba(X_test)[:, 1]

roc_auc = roc_auc_score(y_test, pred_proba)
print("ROC AUC with class_weights for CatBoostClassifier:", roc_auc)

Learning rate set to 0.285946
0:	learn: 0.3596200	total: 353ms	remaining: 5m 52s
1:	learn: 0.2367244	total: 575ms	remaining: 4m 46s
2:	learn: 0.1848167	total: 916ms	remaining: 5m 4s
3:	learn: 0.1631657	total: 1.27s	remaining: 5m 15s
4:	learn: 0.1534461	total: 1.67s	remaining: 5m 33s
5:	learn: 0.1488020	total: 2.04s	remaining: 5m 37s
6:	learn: 0.1456849	total: 2.42s	remaining: 5m 43s
7:	learn: 0.1442393	total: 2.83s	remaining: 5m 50s
8:	learn: 0.1433054	total: 3.21s	remaining: 5m 52s
9:	learn: 0.1425817	total: 3.65s	remaining: 6m 1s
10:	learn: 0.1421007	total: 4.08s	remaining: 6m 6s
11:	learn: 0.1417686	total: 4.53s	remaining: 6m 13s
12:	learn: 0.1414639	total: 5.05s	remaining: 6m 23s
13:	learn: 0.1412631	total: 5.49s	remaining: 6m 26s
14:	learn: 0.1410282	total: 5.95s	remaining: 6m 30s
15:	learn: 0.1408889	total: 6.33s	remaining: 6m 29s
16:	learn: 0.1407480	total: 6.75s	remaining: 6m 30s
17:	learn: 0.1406250	total: 7.21s	remaining: 6m 33s
18:	learn: 0.1404557	total: 7.77s	remaining: 6m